In [2]:
# 데이터 프레임 관련
import pprint
import pandas as pd

# XML 관련
import bs4
from os import name
from lxml import html
import xml.etree.ElementTree as et
from urllib.parse import urlencode, quote_plus, unquote

# API 호출 관련
import json
import requests

# 기타
from tqdm import tqdm
pd.set_option('display.width',1000)

### DF 가져오기

In [109]:
DF = pd.read_csv('./oasis1.csv')

In [110]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    1000 non-null   int64 
 1   bd_mgt_sn     1000 non-null   object
 2   ref_bnm       1000 non-null   object
 3   is_franchise  1000 non-null   int64 
 4   first_date    1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


### 국토교통부_상업업무용 부동산 매매 신고 자료

과거 데이터 수집

In [ ]:
# API 호출
serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'

# 과거 데이터 수집 (~)
for i in range():
    params   = {'serviceKey' : serviceKey,'LAWD_CD' : '11110','DEAL_YMD' : '201512'}
    response = requests.get(url, params=params)

    

# 파일로 저장 (추후 DB 연동 예정)


현재 데이터 수집

In [164]:
# API 호출
url        = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'
serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
params = {
      'serviceKey' : serviceKey
    , 'LAWD_CD'  : '11110'  # 구 단위
    , 'DEAL_YMD' : '202207' # 날짜
}
response = requests.get(url, params=params)
content  = response.text

# item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows    = xml_obj.findAll('item')

# ver 4 (한 줄씩 데이터 프레임으로 만들어서 concat)
# 리스트에 값 담기
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
DF = pd.DataFrame() # 최종 값

# 데이터 수집
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    
    for j in range(0,len(columns)):
        name_list.append(columns[j].name)
        value_list.append(columns[j].text)
    
    row_list = value_list
    df_each  = pd.DataFrame([row_list], columns=name_list)
    DF = pd.concat([DF, df_each])
    
    # 초기화
    name_list  = []
    value_list = []

# CSV 파일로 저장 (추후 DB 연동 예정)
DF.to_excel('국토교통부_상업업무용 부동산 매매 신고 자료(202207).xlsx', encoding='utf-8')

100%|█████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 999.98it/s]


### 1. test

1. 과거 데이터 수집

In [29]:
## 각 컬럼 값 (참고 : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057267)
"""
resultCode : 결과코드
resultMsg : 결과메시지
Deal Amount : 거래금액
Building Area : 건물 전용/연면적
Building Use : 건물주용도
Build Year : 건축년도
Classification of Share Dealing : 지분거래 구분
Deal Year : 계약년도
Plottage : 대지권면적
Dong : 물건 소재 법정동
Sigungu : 물건 소재 시군구
Land Use : 물건 소재 토지 용도지역
Deal Month : 계약월
Building Type : 일반/집합 건물 구분
Deal Day : 계약일
Regional Code : 지역코드
Floor : 거래물건 층(집합건물)
""" 

'\nresultCode : 결과코드\nresultMsg : 결과메시지\nDeal Amount : 거래금액\nBuilding Area : 건물 전용/연면적\nBuilding Use : 건물주용도\nBuild Year : 건축년도\nClassification of Share Dealing : 지분거래 구분\nDeal Year : 계약년도\nPlottage : 대지권면적\nDong : 물건 소재 법정동\nSigungu : 물건 소재 시군구\nLand Use : 물건 소재 토지 용도지역\nDeal Month : 계약월\nBuilding Type : 일반/집합 건물 구분\nDeal Day : 계약일\nRegional Code : 지역코드\nFloor : 거래물건 층(집합건물)\n'

In [5]:
serviceKey_encoding = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
# serviceKey_decoding = 'sJuqUegKT1Q0R4IRDqoJklMK8xDxru+rNu+K5JXRh1mbeH4tFTWwQYWVK3xdbbJCCe14u5bQyl43S28/5TtfrA=='
# key = requests.utils.unquote(serviceKey_encoding) # 디코딩

url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'
params = {'serviceKey' : serviceKey_encoding, 'LAWD_CD' : '11110', 'DEAL_YMD' : '201512' }

response = requests.get(url, params=params)

In [6]:
content = response.text

In [7]:
# 값 확인
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(content))

('<?xml version="1.0" encoding="UTF-8" '
 'standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL '
 'SERVICE.</resultMsg></header><body><items><item><거래금액>120,000</거래금액><거래유형> '
 '</거래유형><건물면적>395.6</건물면적><건물주용도>제2종근린생활</건물주용도><건축년도>2001</건축년도><년>2015</년><대지면적>160.64</대지면적><법정동> '
 '효자동</법정동><시군구>종로구</시군구><용도지역>제2종일반주거</용도지역><월>12</월><유형>일반</유형><일>31</일><중개사소재지> '
 '</중개사소재지><지역코드>11110</지역코드><해제사유발생일> </해제사유발생일><해제여부> '
 '</해제여부></item><item><거래금액>483,000</거래금액><거래유형> '
 '</거래유형><건물면적>937.52</건물면적><건물주용도>제2종근린생활</건물주용도><건축년도>1990</건축년도><년>2015</년><대지면적>503.5</대지면적><법정동> '
 '체부동</법정동><시군구>종로구</시군구><용도지역>제2종일반주거</용도지역><월>12</월><유형>일반</유형><일>15</일><중개사소재지> '
 '</중개사소재지><지역코드>11110</지역코드><해제사유발생일> </해제사유발생일><해제여부> '
 '</해제여부></item><item><거래금액>483,000</거래금액><거래유형> '
 '</거래유형><건물면적>937.52</건물면적><건물주용도>제2종근린생활</건물주용도><건축년도>1990</건축년도><년>2015</년><대지면적>503.5</대지면적><법정동> '
 '체부동</법정동><시군구>종로구</시군구><용도지역>제2종일반주거</용도지역><월>12</월><유형>일반</유형><일>15</일><중개사소재지> '
 '</중개사소재지

In [26]:
# xml을 DataFrame으로 변환
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

In [8]:
# item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')
print(rows)

[<item><거래금액>120,000</거래금액><거래유형> </거래유형><건물면적>395.6</건물면적><건물주용도>제2종근린생활</건물주용도><건축년도>2001</건축년도><년>2015</년><대지면적>160.64</대지면적><법정동> 효자동</법정동><시군구>종로구</시군구><용도지역>제2종일반주거</용도지역><월>12</월><유형>일반</유형><일>31</일><중개사소재지> </중개사소재지><지역코드>11110</지역코드><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>, <item><거래금액>483,000</거래금액><거래유형> </거래유형><건물면적>937.52</건물면적><건물주용도>제2종근린생활</건물주용도><건축년도>1990</건축년도><년>2015</년><대지면적>503.5</대지면적><법정동> 체부동</법정동><시군구>종로구</시군구><용도지역>제2종일반주거</용도지역><월>12</월><유형>일반</유형><일>15</일><중개사소재지> </중개사소재지><지역코드>11110</지역코드><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>, <item><거래금액>483,000</거래금액><거래유형> </거래유형><건물면적>937.52</건물면적><건물주용도>제2종근린생활</건물주용도><건축년도>1990</건축년도><년>2015</년><대지면적>503.5</대지면적><법정동> 체부동</법정동><시군구>종로구</시군구><용도지역>제2종일반주거</용도지역><월>12</월><유형>일반</유형><일>15</일><중개사소재지> </중개사소재지><지역코드>11110</지역코드><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>, <item><거래금액>22,000</거래금액><거래유형> </거래유형><건물면적>31.11</건물면적><건물주용도>판매</건물주용도><건축년도>2007</건축년도><년>2015</년><대지면적> </대지면적><법정동> 종로1가</법정동><시군구>종로구</시군구><

In [9]:
# ver 1
# 리스트에 값 담기
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값

# 데이터 수집
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i == 0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list = []

100%|███████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 17997.01it/s]


In [13]:
# ver 2
# 리스트에 값 담기
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값

# 데이터 수집
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    
    for j in range(0,len(columns)):
        name_list.append(columns[j].name)
        value_list.append(columns[j].text)
    
    row_list.append(value_list)
    value_list = [] # 초기화

100%|████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 3445.88it/s]


In [14]:
df_nm = pd.DataFrame(name_list)
df_nm = df_nm.drop_duplicates()
df_nm = df_nm.reset_index(drop=True)
df_nm = df_nm.transpose()
df_nm.columns = df_nm.iloc[0]

In [15]:
df_nm.columns

Index(['거래금액', '거래유형', '건물면적', '건물주용도', '건축년도', '년', '대지면적', '법정동', '시군구', '용도지역', '월', '유형', '일', '중개사소재지', '지역코드', '해제사유발생일', '해제여부', '층'], dtype='object', name=0)

In [16]:
row_list

[['120,000',
  ' ',
  '395.6',
  '제2종근린생활',
  '2001',
  '2015',
  '160.64',
  ' 효자동',
  '종로구',
  '제2종일반주거',
  '12',
  '일반',
  '31',
  ' ',
  '11110',
  ' ',
  ' '],
 ['483,000',
  ' ',
  '937.52',
  '제2종근린생활',
  '1990',
  '2015',
  '503.5',
  ' 체부동',
  '종로구',
  '제2종일반주거',
  '12',
  '일반',
  '15',
  ' ',
  '11110',
  ' ',
  ' '],
 ['483,000',
  ' ',
  '937.52',
  '제2종근린생활',
  '1990',
  '2015',
  '503.5',
  ' 체부동',
  '종로구',
  '제2종일반주거',
  '12',
  '일반',
  '15',
  ' ',
  '11110',
  ' ',
  ' '],
 ['22,000',
  ' ',
  '31.11',
  '판매',
  '2007',
  '2015',
  ' ',
  ' 종로1가',
  '종로구',
  '일반상업',
  '12',
  '집합',
  '10',
  ' ',
  '11110',
  '-1',
  ' ',
  ' '],
 ['158,000',
  ' ',
  '276.8',
  '숙박',
  '1959',
  '2015',
  '72.8',
  ' 와룡동',
  '종로구',
  '일반상업',
  '12',
  '일반',
  '8',
  ' ',
  '11110',
  ' ',
  ' '],
 ['158,000',
  ' ',
  '276.8',
  '숙박',
  '1959',
  '2015',
  '72.8',
  ' 와룡동',
  '종로구',
  '일반상업',
  '12',
  '일반',
  '8',
  ' ',
  '11110',
  ' ',
  ' '],
 ['247,500',
  ' ',
  '154.04',
  '제2

In [17]:
DF = pd.DataFrame(row_list, columns=[df_nm.columns])

In [18]:
DF

,거래금액,거래유형,건물면적,건물주용도,건축년도,년,대지면적,법정동,시군구,용도지역,월,유형,일,중개사소재지,지역코드,해제사유발생일,해제여부,층
0,"120,000",,395.6,제2종근린생활,2001,2015,160.64,효자동,종로구,제2종일반주거,12,일반,31,,11110,,,None
1,"483,000",,937.52,제2종근린생활,1990,2015,503.5,체부동,종로구,제2종일반주거,12,일반,15,,11110,,,None
2,"483,000",,937.52,제2종근린생활,1990,2015,503.5,체부동,종로구,제2종일반주거,12,일반,15,,11110,,,None
3,"22,000",,31.11,판매,2007,2015,,종로1가,종로구,일반상업,12,집합,10,,11110,-1,,
4,"158,000",,276.8,숙박,1959,2015,72.8,와룡동,종로구,일반상업,12,일반,8,,11110,,,None
5,"158,000",,276.8,숙박,1959,2015,72.8,와룡동,종로구,일반상업,12,일반,8,,11110,,,None
6,"247,500",,154.04,제2종근린생활,1976,2015,105.8,삼청동,종로구,제1종일반주거,12,일반,21,,11110,,,None
7,"160,000",,79.34,제1종근린생활,2015,178.5,삼청동,종로구,제1종일반주거,12,일반,29,,11110,,,None,None
8,"174,000",,323.8,제2종근린생활,1976,2015,191.7,봉익동,종로구,일반상업,12,일반,28,,11110,,,None
9,"162,000",,201.66,제2종근린생활,1961,2015,112.5,인의동,종로구,일반상업,12,일반,3,,11110,,,None


In [20]:
columns[j].name

'거래금액'

In [29]:
DF

,거래금액


In [28]:
# ver 3 (한 줄씩 담기)
# 데이터 프레임에 값 담기
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
name_df  = pd.DataFrame()
value_df = pd.DataFrame()

# 데이터 수집
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    for j in range(0,len(columns)):
        if i == 0:
            name_list.append(columns[j].name)
            df_nm = pd.DataFrame(name_list)
            df_nm = df_nm.drop_duplicates()
            df_nm = df_nm.reset_index(drop=True)
            df_nm = df_nm.transpose()
            df_nm.columns = df_nm.iloc[0]
            DF = pd.DataFrame(columns=[df_nm.columns])
            
        DF.append(columns[j].name)
        DF.append(columns[j].text)
        
    row_list.append(value_list)
    value_list = []


  0%|                                                                                           | 0/36 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_6448\863568316.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF.append(columns[j].name)
  0%|                                                                                           | 0/36 [00:00<?, ?it/s]


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

ver 4

In [163]:
# ver 4 (한 줄씩 데이터 프레임으로 만들어서 concat)
# 리스트에 값 담기
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
DF = pd.DataFrame() # 최종 값

# 데이터 수집
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    
    for j in range(0,len(columns)):
        name_list.append(columns[j].name)
        value_list.append(columns[j].text)
    
    row_list = value_list
    df_each  = pd.DataFrame([row_list], columns=name_list)
    DF = pd.concat([DF, df_each])
    
    # 초기화
    name_list  = []
    value_list = []

100%|█████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 959.76it/s]


In [162]:
DF

,거래금액,거래유형,건물면적,건물주용도,건축년도,년,대지면적,법정동,시군구,용도지역,월,유형,일,중개사소재지,지역코드,해제사유발생일,해제여부,층
0,"120,000",,395.6,제2종근린생활,2001,2015,160.64,효자동,종로구,제2종일반주거,12,일반,31,,11110,,,NaN
0,"120,000",,395.6,제2종근린생활,2001,2015,160.64,효자동,종로구,제2종일반주거,12,일반,31,,11110,,,NaN
0,"483,000",,937.52,제2종근린생활,1990,2015,503.5,체부동,종로구,제2종일반주거,12,일반,15,,11110,,,NaN
0,"483,000",,937.52,제2종근린생활,1990,2015,503.5,체부동,종로구,제2종일반주거,12,일반,15,,11110,,,NaN
0,"22,000",,31.11,판매,2007,2015,,종로1가,종로구,일반상업,12,집합,10,,11110,,,-1
0,"158,000",,276.8,숙박,1959,2015,72.8,와룡동,종로구,일반상업,12,일반,8,,11110,,,NaN
0,"158,000",,276.8,숙박,1959,2015,72.8,와룡동,종로구,일반상업,12,일반,8,,11110,,,NaN
0,"247,500",,154.04,제2종근린생활,1976,2015,105.8,삼청동,종로구,제1종일반주거,12,일반,21,,11110,,,NaN
0,"160,000",,79.34,제1종근린생활,NaN,2015,178.5,삼청동,종로구,제1종일반주거,12,일반,29,,11110,,,NaN
0,"174,000",,323.8,제2종근린생활,1976,2015,191.7,봉익동,종로구,일반상업,12,일반,28,,11110,,,NaN


In [140]:
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값

In [141]:
columns = rows[i].find_all()
columns

[<거래금액>483,000</거래금액>,
 <거래유형> </거래유형>,
 <건물면적>937.52</건물면적>,
 <건물주용도>제2종근린생활</건물주용도>,
 <건축년도>1990</건축년도>,
 <년>2015</년>,
 <대지면적>503.5</대지면적>,
 <법정동> 체부동</법정동>,
 <시군구>종로구</시군구>,
 <용도지역>제2종일반주거</용도지역>,
 <월>12</월>,
 <유형>일반</유형>,
 <일>15</일>,
 <중개사소재지> </중개사소재지>,
 <지역코드>11110</지역코드>,
 <해제사유발생일> </해제사유발생일>,
 <해제여부> </해제여부>]

In [142]:
name_list.append(columns[0].name)
value_list.append(columns[0].text)
name_list

['거래금액']

In [143]:
value_list

['483,000']

In [144]:
row_list.append(value_list)

In [145]:
row_list

[['483,000']]

In [146]:
df_each = pd.DataFrame(row_list, columns=name_list)

In [147]:
df_each

,거래금액
0,"483,000"


In [148]:
DF = pd.concat([DF, df_each])

In [149]:
DF

,거래금액,거래유형,건물면적,건물주용도,건축년도,년,대지면적,법정동,시군구,용도지역,월,유형,일,중개사소재지,지역코드,해제사유발생일,해제여부
0,"120,000",,395.6,제2종근린생활,2001,2015,160.64,효자동,종로구,제2종일반주거,12,일반,31,,11110,,
0,"120,000",,395.6,제2종근린생활,2001,2015,160.64,효자동,종로구,제2종일반주거,12,일반,31,,11110,,
0,"483,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"483,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"483,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# DataFrame으로 변환
# DF = pd.DataFrame(row_list, columns=name_list)
DF = pd.DataFrame(row_list, columns=name_list)
DF.head()

ValueError: 17 columns passed, passed data had 18 columns

In [62]:
# CSV 파일로 저장
DF.to_csv('국토교통부_상업업무용 부동산 매매 신고 자료.csv', encoding='utf-8')

TypeError: to_csv() got an unexpected keyword argument 'drop'

### 상업용부동산 임대동향 조사 통계 조회 서비스

1. 지역별 전환율

In [4]:
# 값 요청
# serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
url = 'https://api.odcloud.kr/api/CommercialRealEstateLeaseTrendSvc/v1/getConvertRate?page=1&perPage=10000&returnType=JSON&cond%5BREGION_CD%3A%3AEQ%5D=1111010100&cond%5BBULD_GBN%3A%3AEQ%5D=1&cond%5BRESEARCH_DATE%3A%3ALT%5D=20220701&cond%5BRESEARCH_DATE%3A%3ALTE%5D=20220701&cond%5BRESEARCH_DATE%3A%3AGT%5D=20220701&cond%5BRESEARCH_DATE%3A%3AGTE%5D=20220701&serviceKey=C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH%2B%2FntypJ7kK2P2g16vLYCJif3TNfO1i1KA%3D%3D'
params = {
  "currentCount": 0,
  "data": [],
  "matchCount": 0,
  "page": 1,
  "perPage": 10000,
  "totalCount": 22766
}
response = requests.get(url, params=params)

# json 형태로 저장
file = json.loads(response.text)

In [11]:
file

{'currentCount': 0,
 'data': [],
 'matchCount': 0,
 'page': 1,
 'perPage': 10000,
 'totalCount': 22766}

2. 지역별 임대가격지수

3. 지역별 분기 소득수익률

4. 지역별 분기 자본수익률

5. 지역별 분기 투자수익률

6. 지역별 유효조소득 중 기타수입 구성비

7. 순영업 소득

8. 지역별 영업경비 구성비

9. 지역별 유효조소득 중 임대수입 구성비

10. 지역별 임대료

11. 층별 임대료

12. 층별 효용비용

13. 지역별 공실률

14. 지역별 연간 소득수익률

15. 지역별 연간 자본수익률

16. 지역별 연간 투자수익률

### 파일 불러와서 합치기

In [136]:
import glob

In [137]:
# 파일 불러오기
tmp_raw = []

for file_name in glob.glob('./data/data/corona/*.csv'):
    tmp = pd.read_csv(file_name)
    tmp_raw.append(tmp)

# 불러온 파일 합치기
df_raw = pd.concat(tmp_raw)

In [139]:
df_raw

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2020-8
0,11,3139,6636,3492,703,1334,1506,5642.0,3865,2714,7769,26568,17500,NaN
0,0,1,9,4,11,11,14,91.0,45,19,70,306,180,NaN
0,0,0,0,0,28,14,12,NaN,125,51,76,937,214,28.0
0,0,1,33,1,6,45,5,60.0,32,22,72,348,180,NaN
0,0,1,11,0,3,17,7,34.0,22,3,27,201,72,NaN
0,0,0,0,25,8,15,6,172.0,73,30,140,352,215,NaN
0,0,0,6,3,10,43,2,73.0,69,22,91,255,273,NaN
0,0,1,25,5,3,4,6,76.0,34,24,230,331,233,NaN
0,0,1,23,7,9,12,18,107.0,59,28,126,408,209,NaN
0,0,1,14,9,3,12,14,82.0,34,26,103,403,158,NaN
